In [9]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
# We use PIL lib to load image from a image path to be consistent with Tensorflow tutorial. You can use Skimage instead like previous weeks.
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

cifar100 = tf.keras.datasets.cifar100 
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

batch_size = 32
img_height = 32
img_width = 32

unique_labels = np.unique(train_labels)
print("Unique labels:", unique_labels)
num_classes = len(unique_labels)

batch_size = 32
img_height = 32
img_width = 32

data_augmentation = Sequential(
  [ 
    layers.RandomFlip("horizontal", 
                      input_shape=(img_height, img_width, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
);

from tensorflow.keras import layers, Model

def residual_block(x, filters, kernel_size, strides=(1, 1), padding='same'):
    # Shortcut branch
    shortcut = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    shortcut = layers.BatchNormalization()(shortcut)

    # Main branch
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size, padding=padding)(x)
    x = layers.BatchNormalization()(x)

    # Add shortcut and main branch
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    x = layers.BatchNormalization()(x)
    return x

# Define the model
inputs = layers.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(16, (7, 7), strides=(1, 1), padding='valid', activation='relu')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

# Add residual blocks
x = residual_block(x, filters=60, kernel_size=(3, 3))
x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

x = residual_block(x, filters=128, kernel_size=(3, 3))
x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)

x = residual_block(x, filters=200, kernel_size=(3, 3))
x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
x = layers.Dropout(0.2)(x) # 0 -> 1 -> dropout_rate=0.5

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(num_classes)(x)

model = Model(inputs, outputs)
model.summary()

# Compile the model


Unique labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_6         │ (None, 32, 32, 3) │          0 │ input_layer_13[0… │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_57 (Conv2D)  │ (None, 26, 26,    │      2,368 │ rescaling_6[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_23    │ (None, 13, 13,    │          0 │ conv2d_57[0][0]   │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_59 (Conv2D)  │ (None, 13, 13,    │      8,700 │ max_pooling2d_23… │
│                     │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        240 │ conv2d_59[0][0]   │
│ (BatchNormalizatio… │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_34 (ReLU)     │ (None, 13, 13,    │          0 │ batch_normalizat… │
│                     │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 13, 13,    │     32,460 │ re_lu_34[0][0]    │
│                     │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_58 (Conv2D)  │ (None, 13, 13,    │      8,700 │ max_pooling2d_23… │
│                     │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        240 │ conv2d_60[0][0]   │
│ (BatchNormalizatio… │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        240 │ conv2d_58[0][0]   │
│ (BatchNormalizatio… │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_17 (Add)        │ (None, 13, 13,    │          0 │ batch_normalizat… │
│                     │ 60)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_35 (ReLU)     │ (None, 13, 13,    │          0 │ add_17[0][0]      │
│                     │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        240 │ re_lu_35[0][0]    │
│ (BatchNormalizatio… │ 60)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_24    │ (None, 6, 6, 60)  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 6, 6, 128) │     69,248 │ max_pooling2d_24… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 6, 128) │        512 │ conv2d_62[0][0]   │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 1,204,544 (4.59 MB)

 Trainable params: 1,201,440 (4.58 MB)

 Non-trainable params: 3,104 (12.12 KB)

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epochs = 15
history_bn_model_filter = model.fit(
  train_images,train_labels,
  validation_data=(test_images, test_labels),
  epochs=epochs,
  verbose=2, # 0 = silent, 1 = progress bar, 2 = one line per epoch
)



# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Accuracy:", test_accuracy)

#accuracy is 0.59

# Plot the training and validation accuracy and loss
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history_bn_model_filter.history['accuracy'], label='Training Accuracy')
plt.plot(history_bn_model_filter.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history_bn_model_filter.history['loss'], label='Training Loss')
plt.plot(history_bn_model_filter.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

Epoch 1/15
1563/1563 - 111s - 71ms/step - accuracy: 0.1327 - loss: 3.7493 - val_accuracy: 0.1815 - val_loss: 3.4568
Epoch 2/15
1563/1563 - 127s - 81ms/step - accuracy: 0.2608 - loss: 2.9724 - val_accuracy: 0.2823 - val_loss: 2.8445
Epoch 3/15
1563/1563 - 117s - 75ms/step - accuracy: 0.3380 - loss: 2.5667 - val_accuracy: 0.3459 - val_loss: 2.5523
Epoch 4/15
1563/1563 - 127s - 82ms/step - accuracy: 0.4007 - loss: 2.2841 - val_accuracy: 0.3809 - val_loss: 2.3996
Epoch 5/15
1563/1563 - 127s - 81ms/step - accuracy: 0.4500 - loss: 2.0441 - val_accuracy: 0.4019 - val_loss: 2.3340
Epoch 6/15
1563/1563 - 116s - 74ms/step - accuracy: 0.4963 - loss: 1.8345 - val_accuracy: 0.4089 - val_loss: 2.3265
Epoch 7/15
1563/1563 - 111s - 71ms/step - accuracy: 0.5438 - loss: 1.6274 - val_accuracy: 0.4129 - val_loss: 2.3308
Epoch 8/15
